# COVID-19 Case Counts from Corona Data Scraper

**[Work in progress]**

This notebook creates a .csv file with cases for ingestion into the Knowledge Graph.

Data source: [Corona Data Scraper](https://coronadatascraper.com/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from pathlib import Path
import pandas as pd
import dateutil

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-9f7418e6-ef5d-4a2d-ae16-29a5a6814849/installation-4.1.0/import


In [4]:
df = pd.read_csv("https://coronadatascraper.com/timeseries.csv.zip",  dtype='str')

##### Remove Unassigned cases (unknown locations)
TODO How should we handle these cases?

In [5]:
df = df[~df['name'].str.startswith("Unassigned cases")]

In [ ]:
df['origLocation'] = df[['country', 'state', 'county', 'city']].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [ ]:
def get_aggregation_level(level):
    if level == 'country':
        return 'Country'
    elif level == 'state':
        return 'Admin1'
    elif level == 'county':
        return 'Admin2'
    elif level == 'city':
        return 'City'

In [ ]:
df['aggregationLevel'] = df['level'].apply(lambda x: get_aggregation_level(x))

In [ ]:
df = df.fillna('')

In [ ]:
df.query("state == 'New York'").head()

In [ ]:
df = df[['date', 'origLocation', 'aggregationLevel', 'cases', 'deaths', 
         'recovered', 'tested', 'hospitalized', 'icu', 'population']]

In [ ]:
df.head()

In [ ]:
df.query('origLocation == "United States,New York"').head()

In [ ]:
df.to_csv(NEO4J_IMPORT / "02b-CDSCases.csv", index=False)